In [58]:
!pip install planet

In [59]:
import os
from shapely import geometry
import pickle
import json
import regex as re
import requests
import pandas as pd
from planet import api
import cv2
import csv
from requests.auth import HTTPBasicAuth
from planet.api import filters
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')
def p(data):
    print(json.dumps(data, indent=2))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
# API Key stored as an env variable
os.environ['PL_API_KEY']='543493b065b14c97aa9b66b26d0ea6f7'
PLANET_API_KEY = os.getenv('PL_API_KEY')
client = api.ClientV1()
client = api.ClientV1(api_key="543493b065b14c97aa9b66b26d0ea6f7")
print(client.auth.value)

543493b065b14c97aa9b66b26d0ea6f7


In [61]:
with open("/content/drive/My Drive/CS_229_project/areas.geojson") as f:
    esri_output  = json.loads(f.read())

In [ ]:
#converts esri multipolygon geojson to planet geojson
all_areas = [item['geometry']['coordinates'] for item in esri_output['features']]
# grab the siteNum for batch downloads
siteNum = [item['properties']['Site_Number'] for item in esri_output['features']]
coords = {}
i = 0
for site in siteNum:
  coords[site] = all_areas[i]
  i += 1
coords.keys()

dict_keys([2, 16, 113, 1103, 1302, 1602, 1701, 2005, 4002, 5005, 7, 5001, 3, 8001, 8005, 5, 12, 27, 1004, 2002, 4001, 4003, 9004, 9010])

In [ ]:
with open("/content/drive/My Drive/CS_229_project/areas2.geojson") as f:
    esri_output  = json.loads(f.read())

In [ ]:
#converts esri multipolygon geojson to planet geojson
all_areas = [item['geometry']['coordinates'] for item in esri_output['features']]
# grab the siteNum for batch downloads
siteNum = [item['properties']['Site_Number'] for item in esri_output['features']]
coords = {}
i = 0
for site in siteNum:
  coords[site] = all_areas[i]
  i += 1
coords.keys()

dict_keys([2022, 9, 10, 16, 500, 1010, 1016, 5001, 6001, 9000, 9001, 9002, 1008, 1201, 6012, 11, 1013, 1014, 1018, 1021, 1025, 2004, 2011, 3001, 4003, 7, 1004, 2002])

In [ ]:
# Grab one study area
geom = { 
  "type": "MultiPolygon",
  "coordinates": all_areas
}

Single Query for detailed information

In [ ]:
# get images that overlap with our AOI 
geo_filter = filters.geom_filter(geom)
# date filters
year = 2020
start_date = datetime(year=year, month=1, day=1)
end_date = datetime(year=year, month=9, day=30)
date_filter = filters.date_range('acquired', gte=start_date, lte = end_date)
# cloud filter
cloud_filter = filters.range_filter('cloud_cover', lte=0.3)
# only visual assets
asset_filter = {'config': ["visual"], 'type': 'AssetFilter'}
# combine our geo, date, cloud filters
and_filter = filters.and_filter(date_filter, cloud_filter, geo_filter, asset_filter)

In [ ]:
# Stats
item_type = ["PSScene3Band"]
req = filters.build_search_request(and_filter, item_type, interval = 'year')
stats = client.stats(req).get()
n = stats['buckets'][0]['count']
p(stats)

{
  "buckets": [
    {
      "count": 13224,
      "start_time": "2020-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}


In [ ]:
item_type = ["PSScene3Band"]
# API request object
req = filters.build_search_request(and_filter, item_type)
res = client.quick_search(req)
data = []
coordsData = []
i = 0
for item in res.items_iter(n):
  i +=1
  for area in coords.keys():
    geom1 = {"type": "Polygon","coordinates": coords.get(area)}
    aoi = geometry.shape(geom1)
    g = geometry.shape(item["geometry"])
    # Optional filter for picture containing aoi
    if g.contains(aoi):
      #print(item["id"])
      context = [area, item['id'], item['properties']['strip_id'], item['properties']['cloud_cover']]
      coordsData.append([area, item['id'], item['geometry']['coordinates']])
      data.append(context)
i

13224

In [ ]:
coords_pd = pd.DataFrame(coordsData, columns= ["site", "file", 'coordinates'])
coords_pd.coordinates.to_list()

[[[[-117.40882041316907, 33.712234230641684],
   [-117.37132202754262, 33.56133991305348],
   [-117.10060336950191, 33.60798690430259],
   [-117.13771289447878, 33.759124112334575],
   [-117.40882041316907, 33.712234230641684]]],
 [[[-117.4330326737897, 33.810314961046686],
   [-117.39540621974822, 33.659441702484145],
   [-117.12443348154777, 33.70613423380675],
   [-117.16158030900166, 33.857260426694545],
   [-117.4330326737897, 33.810314961046686]]],
 [[[-120.3147238159826, 34.650827871845344],
   [-120.36335763895056, 34.465362942754936],
   [-119.98771558285502, 34.39686817454328],
   [-119.93811102758443, 34.58281984722431],
   [-120.3147238159826, 34.650827871845344]]],
 [[[-120.3147238159826, 34.650827871845344],
   [-120.36335763895056, 34.465362942754936],
   [-119.98771558285502, 34.39686817454328],
   [-119.93811102758443, 34.58281984722431],
   [-120.3147238159826, 34.650827871845344]]],
 [[[-116.2103993384133, 33.8807101421349],
   [-116.26217120497051, 33.69959962974139

Batch Search over all sites

In [ ]:
data_pd = pd.DataFrame(data, columns= ["site", "file", 'strip_id', 'cloud_cover'])
data_pd_unique = data_pd.drop_duplicates(subset = ["file"]).sort_values("file")
data_pd_unique

,site,file,strip_id,cloud_cover
1433,7,20200101_164825_0f3c,2991348,0.11
1432,9,20200101_181648_100c,2991721,0.17
1427,1010,20200101_182915_01_106f,2992068,0.07
1428,500,20200101_182917_07_106f,2992068,0.13
1429,5001,20200102_183003_80_106b,2994285,0.12
...,...,...,...,...
14,2022,20200928_184710_50_1060,3765369,0.09
10,1014,20200929_175752_13_2223,3767981,0.03
9,2011,20200929_175754_34_2223,3767981,0.19
0,9001,20200929_181714_65_1063,3768952,0.12


In [ ]:
with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite/file2siteGroup2lte30" + str(year) + ".txt", "wb") as fp:
  pickle.dump(data_pd_unique,fp)

In [ ]:
path = '/content/drive/My Drive/CS_229_project/img_data/cloudy/'
f = os.listdir(path)
len(f)

1527

In [ ]:
with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite/file2siteGroup2lte302017.txt", "rb") as fp:
  l2017 = pickle.load(fp)
with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite/file2siteGroup2lte302018.txt", "rb") as fp:
  l2018 = pickle.load(fp)
with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite/file2siteGroup2lte302019.txt", "rb") as fp:
  l2019 = pickle.load(fp)
with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite/file2siteGroup2lte302020.txt", "rb") as fp:
  l2020 = pickle.load(fp)

l = l2017.append(l2018).append(l2019).append(l2020)
l

with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite/file2siteGroup2lte30.txt", "wb") as fp:
  pickle.dump(l,fp)

In [ ]:
l

,site,file,strip_id,cloud_cover
493,1016,20170106_174403_0e20,352326,0.00
492,2004,20170106_182159_0c13,353704,0.02
491,2002,20170116_175224_0e14,364648,0.02
490,2004,20170116_175903_0e19,364732,0.00
489,2004,20170125_180016_0e20,380850,0.00
...,...,...,...,...
14,2022,20200928_184710_50_1060,3765369,0.09
10,1014,20200929_175752_13_2223,3767981,0.03
9,2011,20200929_175754_34_2223,3767981,0.19
0,9001,20200929_181714_65_1063,3768952,0.12


In [97]:
path = '/content/drive/My Drive/CS_229_project/img_data/county_site/'
f = os.listdir(path)
print(f[0:5])
len(f)

['59_5001_20190405_185026_37_106c_3B_Visual_clip.jpg', '59_5001_20181014_180625_1014_3B_Visual_clip.jpg', '59_5001_20190902_184228_93_1067_3B_Visual_clip.jpg', '59_5001_20181007_180623_1005_3B_Visual_clip.jpg', '59_5001_20200517_182913_87_1067_3B_Visual_clip.jpg']


7202

In [94]:
fset = set(f)
len(fset)

7202

In [106]:
l = []
d = {}
df = pd.DataFrame(columns=['id', 'file'])
for i in f:
  key = i.split('_')[0] + "_" + i.split('_')[1]
  l.append(key)
  d.setdefault(key, [])
  d[key].append(i)
  n = len(i)
  strp = len("_3B_Visual_clip.jpg")
  file = i[len(key) + 1:n-strp]
  df.loc[i] = [key] + [file] 

In [107]:
df.head()
df[df.index == '59_5001_20191108_183919_80_106a_3B_Visual_clip.jpg']

,id,file
59_5001_20191108_183919_80_106a_3B_Visual_clip.jpg,59_5001,20191108_183919_80_106a


In [ ]:
with open("/content/drive/My Drive/CS_229_project/img_data/file2siteWClouds.txt", "rb") as fp:
  clouds = pickle.load(fp)
clouds

,site,file,strip_id,cloud_cover
519,4001,20170108_174732_0e0d,355793,0.04
494,2005,20170108_174959_0e3a,355617,0.05
518,9010,20170118_174436_0e0e,366745,0.30
517,2005,20170126_175021_0e14,382522,0.04
516,7,20170131_174905_0e14,390962,0.03
...,...,...,...,...
1,2002,20200929_174652_39_222b,3768214,0.00
6,8005,20200929_174654_60_222b,3768214,0.00
4,1602,20200929_181638_103b,3769024,0.01
5,4002,20200929_181641_103b,3769024,0.01


In [78]:
df[df['filefull'] == '20190405_185026_37_106c_3B_Visual_clip.jpg']

,id,file,filefull
59_5001_20190405_185026_37_106c_3B_Visual_clip.jpg,59_5001,20190405_185026_37_106,20190405_185026_37_106c_3B_Visual_clip.jpg


In [ ]:
df2.fillna({'cloud_cover':0.3})

,id,file,site,strip_id,cloud_cover
0,59_5001,20190405_185026_37_106,NaN,NaN,0.30
1,59_5001,20181014_180625_1014,5001.0,1767245,0.87
2,59_5001,20190902_184228_93_1067,5001.0,2646834,0.18
3,59_5001,20181007_180623_1005,5001.0,1751570,0.19
4,59_5001,20200517_182913_87_1067,5001.0,3405714,0.12
...,...,...,...,...,...
7197,83_4003,20170615_175821_1010,NaN,NaN,0.30
7198,83_4003,20190516_185741_85_106e,NaN,NaN,0.30
7199,83_4003,20191102_181448_0e20,NaN,NaN,0.30
7200,83_4003,20200430_210627_0f02,NaN,NaN,0.30


In [108]:
df2 = df
df2['file'] = df['file'].str.lower()
df2 = df2.merge(clouds, how = 'left', on = 'file')
df2 = df2.fillna({'cloud_cover':0.3})
with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite.txt", "wb") as fp:
  pickle.dump(df2,fp)
df2

,id,file,site,strip_id,cloud_cover
0,59_5001,20190405_185026_37_106c,5001.0,2260937,0.18
1,59_5001,20181014_180625_1014,5001.0,1767245,0.87
2,59_5001,20190902_184228_93_1067,5001.0,2646834,0.18
3,59_5001,20181007_180623_1005,5001.0,1751570,0.19
4,59_5001,20200517_182913_87_1067,5001.0,3405714,0.12
...,...,...,...,...,...
7197,83_4003,20170615_175821_1010,NaN,NaN,0.30
7198,83_4003,20190516_185741_85_106e,NaN,NaN,0.30
7199,83_4003,20191102_181448_0e20,NaN,NaN,0.30
7200,83_4003,20200430_210627_0f02,NaN,NaN,0.30


In [110]:
df2['filefull'] = df2['id'] + '_' + df2['file'] + '_3B_Visual_clip.jpg'
df2[df2['filefull'] == '59_5001_20191108_183919_80_106a_3B_Visual_clip.jpg']

,id,file,site,strip_id,cloud_cover,filefull
14,59_5001,20191108_183919_80_106a,5001.0,2822496,0.12,59_5001_20191108_183919_80_106a_3B_Visual_clip...


In [ ]:
lset = set(l)
print(len(lset))
lset

52


{'111_1004',
 '111_2002',
 '111_3001',
 '111_7',
 '111_9',
 '37_1103',
 '37_113',
 '37_1201',
 '37_1302',
 '37_16',
 '37_1602',
 '37_1701',
 '37_2',
 '37_2005',
 '37_4002',
 '37_5005',
 '37_6012',
 '59_2022',
 '59_5001',
 '59_7',
 '65_10',
 '65_1010',
 '65_1016',
 '65_16',
 '65_3',
 '65_500',
 '65_5001',
 '65_6001',
 '65_8005',
 '65_9001',
 '71_1004',
 '71_12',
 '71_2002',
 '71_27',
 '71_4001',
 '71_4003',
 '71_5',
 '71_8001',
 '71_9002',
 '71_9004',
 '71_9010',
 '73_1008',
 '83_1013',
 '83_1014',
 '83_1018',
 '83_1021',
 '83_1025',
 '83_11',
 '83_2004',
 '83_2011',
 '83_4003',
 '83_9000'}

In [ ]:
#for i in f:
  #key = i.split('_')[0]
  #rename = d[int(key)] + i[len(key):]
  #print(path + rename)
  #os.rename(path + i, path + rename)

In [ ]:
map = pd.read_csv('/content/drive/My Drive/CS_229_project/group1.tsv', delimiter = "\t")
map['combined'] = map['county'].astype(str) + "_" + map['site'].astype(str)
map.sort_values(by='site')

,county,site,combined
0,37,2,37_2
12,65,3,65_3
15,71,5,71_5
10,59,7,59_7
16,71,12,71_12
1,37,16,37_16
17,71,27,71_27
2,37,113,37_113
18,71,1004,71_1004
3,37,1103,37_1103


In [ ]:
sites = map['site'].tolist()
text = map['combined'].tolist()
d = {sites[i]:text[i] for i in range(len(sites))}
d

{2: '37_2',
 3: '65_3',
 5: '71_5',
 7: '59_7',
 12: '71_12',
 16: '37_16',
 27: '71_27',
 113: '37_113',
 1004: '71_1004',
 1103: '37_1103',
 1302: '37_1302',
 1602: '37_1602',
 1701: '37_1701',
 2002: '71_2002',
 2005: '37_2005',
 4001: '71_4001',
 4002: '37_4002',
 4003: '71_4003',
 5001: '59_5001',
 5005: '37_5005',
 8001: '71_8001',
 8005: '65_8005',
 9004: '71_9004',
 9010: '71_9010'}

In [ ]:
im11 = cv2.imread(path + '8005_20170309_174810_0e26_3B_Visual_clip.jpg')
im12 = cv2.imread(path + '8005_20200330_183651_40_1058_3B_Visual_clip.jpg')
im13 = cv2.imread(path + '8005_20200919_174814_98_2257_3B_Visual_clip.jpg')
im14 = cv2.imread(path + '8005_20200317_174350_86_2257_3B_Visual_clip.jpg')
im15 = cv2.imread(path + '8005_20190723_182551_55_1059_3B_Visual_clip.jpg')

im21 = cv2.imread(path + '16_20200611_174641_66_2259_3B_Visual_clip.jpg')
im22 = cv2.imread(path + '16_20170726_175014_0f25_3B_Visual_clip.jpg')
im23 = cv2.imread(path + '16_20170829_175200_102c_3B_Visual_clip.jpg')
im24 = cv2.imread(path + '16_20200805_182409_37_1065_3B_Visual_clip.jpg')
im25 = cv2.imread(path + '16_20200411_180002_0e20_3B_Visual_clip.jpg')

im31 = cv2.imread(path + '3_20190528_184641_13_1068_3B_Visual_clip.jpg')
im32 = cv2.imread(path + '3_20200727_174811_31_2271_3B_Visual_clip.jpg')
im33 = cv2.imread(path + '3_20200317_174352_89_2257_3B_Visual_clip.jpg')
im34 = cv2.imread(path + '3_20190611_182020_69_1059_3B_Visual_clip.jpg')
im35 = cv2.imread(path + '3_20190629_182217_78_1057_3B_Visual_clip.jpg')

im41 = cv2.imread(path + '8001_20200204_183232_98_1060_3B_Visual_clip.jpg')
im42 = cv2.imread(path + '8001_20200508_182352_70_1065_3B_Visual_clip.jpg')
im43 = cv2.imread(path + '8001_20200923_174431_83_220b_3B_Visual_clip.jpg')
im44 = cv2.imread(path + '8001_20180905_175414_1020_3B_Visual_clip.jpg')
im45 = cv2.imread(path + '8001_20200319_174231_62_2259_3B_Visual_clip.jpg')



In [ ]:
im_1 = cv2.hconcat([im11, im12, im13, im14, im15, im21, im22, im23, im24, im25])
im_3 = cv2.hconcat([im31, im32, im33, im34, im35, im41, im42, im43, im44, im45])
im_all = cv2.vconcat([im_1, im_3])

cv2.imwrite('/content/combined.jpg', im_all)

True